# 4. Trip details

# Observations

- `tripid` attribute exists in vehicle trip details API but does not exist in route details API. Ex: `tripid = 71830930`

## Issue 1. Vehicle is assigned to more than one route

Vehicles on some occassions are assigned to more than one route. Ex: Vehicle `KA57F5808` is assigned to two routes, `routeno = 210-NA` and `routeno = D33-PPLO`.

```json
"LiveLocation": [
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "210-NA",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    },
    {
        "latitude": 12.909809,
        "longitude": 77.536422,
        "location": "Depot-33 Poornapragna layout (Towards Depot-33 (Poornapragna layout))",
        "lastrefreshon": "17-09-2025 23:19:34",
        "nextstop": null,
        "previousstop": "Arehalli (Towards Kengeri)",
        "vehicleid": 27211,
        "vehiclenumber": "KA57F5808",
        "routeno": "D33-PPLO",
        "servicetypeid": 72,
        "servicetype": "Non AC/Ordinary",
        "heading": 210.18,
        "responsecode": 200,
        "trip_status": 1,
        "lastreceiveddatetimeflag": 1
    }
]
```

## Issue 2. Live location is missing

Live location was empty for about `2.5%` of vehicles (`185` out of `7,247` vehicles) when run on `2025-09-17`. Ex: Vehicle ID  `28622` with registration number `KA01AR4181`.

```json
{
    "RouteDetails": [],
    "LiveLocation": [],
    "Message": "No Records Found",
    "Issuccess": true,
    "exception": null,
    "RowCount": 0,
    "responsecode": 200
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.trip_details

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm
import geojson
import random
import tarfile

import requests
import pandas as pd
pd.set_option('display.max_columns', None)

from fastcore.all import Path
from nbdev.config import get_config

from traffic_data_bengaluru.utils import *
from traffic_data_bengaluru.bmtc.vehicles import get_vehicles

In [ ]:
#| export
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory() / "bmtc"

# Fetch

In [ ]:
# | export
def fetch_trip_details(vehicle_id: int, sleep_duration: float = 0.1):
    """Fetch trip details for a given vehicle ID from the BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/VehicleTripDetails_v2"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"
    }

    payload = json.dumps({"vehicleId": int(vehicle_id)})
    try:
        response = requests.post(url, headers=headers, data=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        print("Response text:", getattr(e.response, "text", None))
        return None

In [ ]:
#| hide
#| eval: false

df_vehicles = get_vehicles(data_directory)
trip_details = fetch_trip_details(df_vehicles.sample().iloc[0]['vehicle_id'])
trip_details

# Extract

## Dataset 1. Live Location

In [ ]:
# | export
def extract_live_location(trip_details):
    """Extract live location from trip details."""
    try:
        locations = trip_details['LiveLocation']
    except TypeError as e:
        # When trip_details is None.
        locations = []

    # When there are more than one live locations, it's mostly because of the vehicle assigned to more than one route at a time.
    # We could use the route details and live location to determine which is the right route that the vehicle is running on.
    return locations

In [ ]:
#| hide
#| eval: false

live_location = extract_live_location(trip_details)

In [ ]:
# | export
def get_live_locations(directory: Path):
    """Extract live location for all trip details in a directory."""
    latest = get_latest_file(data_directory / 'raw' / 'vehicle_trip_details')
    files = extract_files(latest)
    return files

    live_locations = []
    for filepath in tqdm(files, total = files.__len__(), desc = 'Extracting live locations'):
        live_locations += extract_live_location(read_file(filepath))
    live_locations = pd.DataFrame(live_locations)
    return live_locations

In [ ]:
#| hide
#| eval: false

directory = get_latest_file(data_directory / 'raw' / 'vehicle_trip_details')
df_live_locations = get_live_locations(directory)

print(df_live_locations.shape)
df_live_locations.head(3)

## Dataset 2. Route Details

In [ ]:
# | export
def extract_route_detail(trip_detail):
    try:
        details = trip_detail['RouteDetails']
    except TypeError as e:
        details = []
    return details

In [ ]:
#| hide
#| eval: false
def collect_route_details(directory: Path):
    route_details = []
    for filepath in tqdm(directory.ls(), total = directory.ls().__len__(), desc = 'Extracting route details'):
        with open(filepath) as f:
            trip_detail = json.load(f)
            
            route_detail = extract_route_detail(trip_detail)
            route_details += route_detail
    route_details = pd.DataFrame(route_details)
    return route_details

In [ ]:
#| hide
#| eval: false

directory = get_latest_directory(data_directory / 'raw' / 'vehicle_trip_details')
df_route_details = collect_route_details(directory)

filepath = data_directory / 'latest' / 'route_details.csv'
filepath.parent.mkdir(parents=True, exist_ok=True)
df_route_details.to_csv(filepath, index=False)

print(df_route_details.shape)
df_route_details.head(3)

In [ ]:
#| hide
#| eval: false

sample_vehicleid = df_live_locations[df_live_locations['vehicleid'].notna()].sample(1).iloc[0]['vehicleid']
columns = ['busno', 'stationname', 'sch_arrivaltime', 'actual_arrivaltime']
df = df_route_details[df_route_details['vehicleid'] == sample_vehicleid].reset_index(drop=True)[columns]

# Keep the raw values as is, and calculate diffs on the fly when needed
def calc_time_diff(series):
    times = pd.to_datetime(series, format="%H:%M", errors="coerce")
    return times.diff().dt.total_seconds()

df["sch_diff"] = calc_time_diff(df["sch_arrivaltime"])
df["actual_diff"] = calc_time_diff(df["actual_arrivaltime"])

df

# Task

In [ ]:
# | export
def task_fetch_trip_details(data_directory: Path):
    folder_name = f'{str(int(datetime.datetime.now().timestamp()))}'
    directory = data_directory / 'raw' / 'vehicle_trip_details' / folder_name
    directory.mkdir(exist_ok=True, parents=True)

    logger.info("Fetching vehicle trip details ...")
    df_vehicles = get_vehicles(data_directory)
    for index, row in tqdm(df_vehicles.iterrows(), total = df_vehicles.shape[0], desc = 'Fetching vehicle trip details'):
        trip_details = fetch_trip_details(vehicle_id = row['vehicle_id']) 
        with open(directory / f"{row['vehicle_id']}.json", "w") as f:
            json.dump(trip_details, f, indent = 4)

    tar_path = directory.with_suffix('.tar.gz')
    with tarfile.open(tar_path, "w:gz") as tar:
        tar.add(directory, arcname=directory.name)

    logger.info(f"Compressed {directory} into {tar_path}")

In [ ]:
#| hide
#| eval: false

task_fetch_trip_details(data_directory=data_directory)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()